In [1]:
import scanpy as sc
import stlearn as st
import numpy as np
import csv
import pickle
from scipy import sparse
import scipy.io as sio
import matplotlib
matplotlib.use('Agg')
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_hex, rgb2hex
from typing import List
import qnorm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from collections import defaultdict
import pandas as pd
import gzip
from kneed import KneeLocator
import copy 
#sys.path.append("/home/gw/code/utility/altairThemes/")
#if True:  # In order to bypass isort when saving
#    import altairThemes
#import altairThemes
import altair as alt
import argparse

spot_diameter = 89.43 #pixels

In [2]:
data_name = 'PDAC_64630'

In [3]:
# read filtered matrix
adata_h5 = st.Read10X(path="/Users/victoriagao/local_docs/NEST/input", count_file='filtered_feature_bc_matrix.h5')
adata_h5


/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/victoriagao/anaconda3/envs/NEST_try/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 1406 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [4]:
adata_h5.X

<1406x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 2345152 stored elements in Compressed Sparse Row format>

In [5]:
# filter out the genes that are not expressed in any cell
sc.pp.filter_genes(adata_h5, min_cells=1)
adata_h5

AnnData object with n_obs × n_vars = 1406 × 19523
    obs: 'in_tissue', 'array_row', 'array_col', 'sum_counts', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'spatial'
    obsm: 'spatial'

In [6]:
# get gene identifier - names
gene_ids = list(adata_h5.var_names)
gene_ids

['AL627309.1',
 'AL627309.5',
 'AP006222.2',
 'LINC01409',
 'LINC01128',
 'LINC00115',
 'FAM41C',
 'LINC02593',
 'SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'PERM1',
 'AL645608.7',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'LINC01342',
 'AL390719.2',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'C1QTNF12',
 'AL162741.1',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'INTS11',
 'CPTP',
 'TAS1R3',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20-AS1',
 'MRPL20',
 'AL391244.2',
 'ANKRD65',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'TMEM240',
 'SSU72',
 'AL645728.1',
 'FNDC10',
 'AL691432.2',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'SLC35E2A',
 'NADK',
 'GNB1',
 'AL109917.1',
 'CALML6',
 'TMEM52',
 'AL391845.2',
 'GABRD',
 'PRKCZ',
 'PRKCZ-AS1',
 'FAAP20',
 'SKI',
 'AL590822.3',
 'MORN1',
 'AL589739.1',
 'AL513477.2',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'AL139246.5',
 'TNFRSF14-AS1',
 'TNFRSF14',
 'AL139246.3',
 'PRXL2B',
 'MMEL1',
 'TTC34',
 'AC

In [7]:
# access coordinates stored in adata_ha using the key spatial
coordinates = adata_h5.obsm['spatial']
coordinates

array([[ 9283, 12018],
       [11004, 10622],
       [ 8053,  6963],
       ...,
       [12040, 10617],
       [ 9801, 12115],
       [ 3230,  9762]])

In [8]:
# all x-position of cells
coordinates[:,0]
# all y-position of cells
coordinates[0,:]

array([ 9283, 12018])

In [10]:
# get cell identifier - barcode
cell_barcode = np.array(adata_h5.obs.index)
temp = adata_h5.X

In [11]:
cell_barcode

array(['AAACCGGGTAGGTACC-1', 'AAACCGTTCGTCCAGG-1', 'AAACGAGACGGTTGAT-1',
       ..., 'TTGTTTCACATCCAGG-1', 'TTGTTTCCATACAACT-1',
       'TTGTTTGTGTAAATTC-1'], dtype=object)

In [32]:
## get the cell vs gene matrix ##################
temp = qnorm.quantile_normalize(np.transpose(sparse.csr_matrix.toarray(temp)))
adata_X = np.transpose(temp)

cell_vs_gene = copy.deepcopy(adata_X)
## get the cell vs gene matrix ##################

In [12]:
##################### make cell metadata: barcode_info ###################################
 
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0]) # last entry will hold the component number later
    i=i+1
    

In [13]:
barcode_serial

{'AAACCGGGTAGGTACC-1': 0,
 'AAACCGTTCGTCCAGG-1': 1,
 'AAACGAGACGGTTGAT-1': 2,
 'AAACTGCTGGCTCCAA-1': 3,
 'AAAGACCCAAGTCGCG-1': 4,
 'AAAGGCTCTCGCGCCG-1': 5,
 'AAAGGGATGTAGCAAG-1': 6,
 'AAAGGGCAGCTTGAAT-1': 7,
 'AAAGTAGCATTGCTCA-1': 8,
 'AAAGTGTGATTTATCT-1': 9,
 'AAATACCTATAAGCAT-1': 10,
 'AAATAGGGTGCTATTG-1': 11,
 'AAATCGTGTACCACAA-1': 12,
 'AAATGGTCAATGTGCC-1': 13,
 'AAATTAACGGGTAGCT-1': 14,
 'AAATTGATAGTCCTTT-1': 15,
 'AAATTTGCGGGTGTGG-1': 16,
 'AACAACTGGTAGTTGC-1': 17,
 'AACAATTACTCTACGC-1': 18,
 'AACACACGCTCGCCGC-1': 19,
 'AACAGGATGGGCCGCG-1': 20,
 'AACCAAGACTTCTCTG-1': 21,
 'AACCCAGAGACGGAGA-1': 22,
 'AACCGCTAAGGGATGC-1': 23,
 'AACCTTTAAATACGGT-1': 24,
 'AACGATAATGCCGTAG-1': 25,
 'AACGCGGTCTCCAGCC-1': 26,
 'AACGCTGTTGCTGAAA-1': 27,
 'AACGGACGTACGTATA-1': 28,
 'AACGGCCATCTCCGGT-1': 29,
 'AACGTACTGTGGGTAC-1': 30,
 'AACGTCAGACTAGTGG-1': 31,
 'AACGTGCGAAAGTCTC-1': 32,
 'AACTAGGCTTGGGTGT-1': 33,
 'AACTCAAGTTAATTGC-1': 34,
 'AACTCTCAGTGTGCTC-1': 35,
 'AACTTGCCCGTATGCA-1': 36,
 'AAGACTGCA

In [31]:
# a list of lists of: barcode, coordinate x position, coordinate y position, 0
first_values = [sublist[0] for sublist in barcode_info]
first_values

['AAACCGGGTAGGTACC-1',
 'AAACCGTTCGTCCAGG-1',
 'AAACGAGACGGTTGAT-1',
 'AAACTGCTGGCTCCAA-1',
 'AAAGACCCAAGTCGCG-1',
 'AAAGGCTCTCGCGCCG-1',
 'AAAGGGATGTAGCAAG-1',
 'AAAGGGCAGCTTGAAT-1',
 'AAAGTAGCATTGCTCA-1',
 'AAAGTGTGATTTATCT-1',
 'AAATACCTATAAGCAT-1',
 'AAATAGGGTGCTATTG-1',
 'AAATCGTGTACCACAA-1',
 'AAATGGTCAATGTGCC-1',
 'AAATTAACGGGTAGCT-1',
 'AAATTGATAGTCCTTT-1',
 'AAATTTGCGGGTGTGG-1',
 'AACAACTGGTAGTTGC-1',
 'AACAATTACTCTACGC-1',
 'AACACACGCTCGCCGC-1',
 'AACAGGATGGGCCGCG-1',
 'AACCAAGACTTCTCTG-1',
 'AACCCAGAGACGGAGA-1',
 'AACCGCTAAGGGATGC-1',
 'AACCTTTAAATACGGT-1',
 'AACGATAATGCCGTAG-1',
 'AACGCGGTCTCCAGCC-1',
 'AACGCTGTTGCTGAAA-1',
 'AACGGACGTACGTATA-1',
 'AACGGCCATCTCCGGT-1',
 'AACGTACTGTGGGTAC-1',
 'AACGTCAGACTAGTGG-1',
 'AACGTGCGAAAGTCTC-1',
 'AACTAGGCTTGGGTGT-1',
 'AACTCAAGTTAATTGC-1',
 'AACTCTCAGTGTGCTC-1',
 'AACTTGCCCGTATGCA-1',
 'AAGACTGCAAGCTACT-1',
 'AAGAGATGAATCGGTA-1',
 'AAGAGCTCTTTATCGG-1',
 'AAGAGGATGTACGCGA-1',
 'AAGAGGCATGGATCGC-1',
 'AAGATGGCACCGGACC-1',
 'AAGCACCCT

In [22]:
# seeing by row/each cell
barcode_info[2]


['AAACGAGACGGTTGAT-1', 8053, 6963, 0]

In [23]:
# make barcode_info into a dictionary
barcode_info_dict = {item[0]: [item[1], item[2]] for item in barcode_info}


In [24]:
barcode_info_dict

{'AAACCGGGTAGGTACC-1': [9283, 12018],
 'AAACCGTTCGTCCAGG-1': [11004, 10622],
 'AAACGAGACGGTTGAT-1': [8053, 6963],
 'AAACTGCTGGCTCCAA-1': [9785, 8146],
 'AAAGACCCAAGTCGCG-1': [3750, 10057],
 'AAAGGCTCTCGCGCCG-1': [11516, 9329],
 'AAAGGGATGTAGCAAG-1': [6161, 8658],
 'AAAGGGCAGCTTGAAT-1': [6176, 12230],
 'AAAGTAGCATTGCTCA-1': [10837, 12111],
 'AAAGTGTGATTTATCT-1': [9600, 5468],
 'AAATACCTATAAGCAT-1': [10129, 7946],
 'AAATAGGGTGCTATTG-1': [11161, 6949],
 'AAATCGTGTACCACAA-1': [9617, 9238],
 'AAATGGTCAATGTGCC-1': [7719, 9743],
 'AAATTAACGGGTAGCT-1': [7889, 9047],
 'AAATTGATAGTCCTTT-1': [11526, 11711],
 'AAATTTGCGGGTGTGG-1': [10144, 11518],
 'AACAACTGGTAGTTGC-1': [6860, 10639],
 'AACAATTACTCTACGC-1': [7040, 12325],
 'AACACACGCTCGCCGC-1': [10809, 5562],
 'AACAGGATGGGCCGCG-1': [7037, 11730],
 'AACCAAGACTTCTCTG-1': [8249, 12419],
 'AACCCAGAGACGGAGA-1': [4617, 10947],
 'AACCGCTAAGGGATGC-1': [9947, 5665],
 'AACCTTTAAATACGGT-1': [11338, 8040],
 'AACGATAATGCCGTAG-1': [11163, 7545],
 'AACGCGGTCTCCAG

In [27]:
barcode_info_dict['AAACCGGGTAGGTACC-1'][0]

9283

In [70]:
####### load pathologist's annotations ##############################################

if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]
        

In [71]:
barcode_type

{'AAACCGGGTAGGTACC-1': 'tumor',
 'AAACCGTTCGTCCAGG-1': 'tumor',
 'AAACGAGACGGTTGAT-1': 'stroma_deserted',
 'AAACTGCTGGCTCCAA-1': 'stroma_deserted',
 'AAAGACCCAAGTCGCG-1': 'stroma_deserted',
 'AAAGGCTCTCGCGCCG-1': 'stroma_deserted',
 'AAAGGGATGTAGCAAG-1': 'tumor',
 'AAAGGGCAGCTTGAAT-1': 'stroma_deserted',
 'AAAGTAGCATTGCTCA-1': 'stroma_deserted',
 'AAAGTGTGATTTATCT-1': 'stroma_deserted',
 'AAATACCTATAAGCAT-1': 'stroma_deserted',
 'AAATAGGGTGCTATTG-1': 'tumor',
 'AAATCGTGTACCACAA-1': 'tumor',
 'AAATGGTCAATGTGCC-1': 'stroma_deserted',
 'AAATTAACGGGTAGCT-1': 'stroma_deserted',
 'AAATTGATAGTCCTTT-1': 'stroma_deserted',
 'AAATTTGCGGGTGTGG-1': 'stroma_deserted',
 'AACAACTGGTAGTTGC-1': 'stroma_deserted',
 'AACAATACATTGTCGA-1': 'Artifact',
 'AACAATTACTCTACGC-1': 'tumor',
 'AACACACGCTCGCCGC-1': 'stroma_deserted',
 'AACAGCTGTGTGGCAA-1': 'Artifact',
 'AACAGGATGGGCCGCG-1': 'stroma_deserted',
 'AACCAAGACTTCTCTG-1': 'tumor',
 'AACCCAGAGACGGAGA-1': 'stroma_deserted',
 'AACCGAGCTTGGTCAT-1': 'Artifact',

In [28]:
len(barcode_info)

1406